In [4]:
import os
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

# Set paths
input_root = r'E:\skin dataset bro\PH2Dataset\PH2 Dataset images'
output_root = r'E:\skin dataset bro\resized'
os.makedirs(output_root, exist_ok=True)

# Define transform for dermoscopic images
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # For grayscale; update for RGB if needed
])

# Define transform for masks (no normalization)
mask_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()  # Just convert to tensor [0, 1]
])

def process_and_save(image_path, save_path, transform):
    if not os.path.exists(image_path): return
    img = Image.open(image_path).convert("RGB")
    img = transform(img)
    img_pil = transforms.ToPILImage()(img)
    img_pil.save(save_path)

# LOOP starts here
for lesion_id in tqdm(os.listdir(input_root), desc="Processing PH2 dataset"):
    lesion_folder = os.path.join(input_root, lesion_id)
    if not os.path.isdir(lesion_folder):
        continue

    img_path = os.path.join(lesion_folder, f"{lesion_id}_Dermoscopic_Image", f"{lesion_id}.bmp")
    lesion_mask_path = os.path.join(lesion_folder, f"{lesion_id}_lesion", f"{lesion_id}_lesion.bmp")

    # Get ROI mask if exists
    roi_mask_path = None
    roi_dir = os.path.join(lesion_folder, f"{lesion_id}_roi")
    if os.path.exists(roi_dir):
        roi_mask_path = next((os.path.join(roi_dir, f) for f in os.listdir(roi_dir) if f.endswith(".bmp")), None)

    # Output paths
    out_img = os.path.join(output_root, f"{lesion_id}_image.png")
    out_lesion = os.path.join(output_root, f"{lesion_id}_lesion.png")
    out_roi = os.path.join(output_root, f"{lesion_id}_roi.png")

    # Process and save
    process_and_save(img_path, out_img, image_transform)
    process_and_save(lesion_mask_path, out_lesion, mask_transform)
    if roi_mask_path:
        process_and_save(roi_mask_path, out_roi, mask_transform)


Processing PH2 dataset:   0%|          | 0/200 [00:00<?, ?it/s]

Processing PH2 dataset: 100%|██████████| 200/200 [00:19<00:00, 10.22it/s]
